# contributors:
simran4@wisc.edu, rgundavarapu@wisc.edu

# PART 1

In [1]:
!curl https://pages.cs.wisc.edu/~harter/cs639/data/hdma-wi-2021.csv > wi-2021.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  166M  100  166M    0     0  12.8M      0  0:00:13  0:00:13 --:--:-- 10.4M


In [2]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp wi-2021.csv hdfs://main:9000/single.csv

In [3]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp wi-2021.csv hdfs://main:9000/double.csv

In [4]:
!hdfs dfs -du -h hdfs://main:9000/

166.8 M  333.7 M  hdfs://main:9000/double.csv
166.8 M  166.8 M  hdfs://main:9000/single.csv


# PART 2

In [5]:
import requests

In [7]:
blocks = {}
for i in range(167):
    url = "http://main:9870/webhdfs/v1/single.csv?op=OPEN&offset="+str(i*1024*1024)
    resp = requests.get(url, allow_redirects=False)
    keyval = resp.headers["Location"].split('?')[0]
    if keyval in blocks.keys():
        blocks[keyval] = blocks[keyval] + 1
    else:
        blocks[keyval] = 1
blocks

{'http://a7cc8c8e1c27:9864/webhdfs/v1/single.csv': 92,
 'http://87931e77d04b:9864/webhdfs/v1/single.csv': 75}

# PART 3

In [8]:
import io
import json
class hdfsFile(io.RawIOBase):
    def __init__(self, path):
        self.path = f"http://main:9870/webhdfs/v1/{path}"
        self.offset = 0
        resp=requests.get("http://main:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
        resp=(resp.content).decode()
        resp=json.loads(resp) # extracting the length by converting byte to string to dict
        self.length = resp['FileStatus']['length']
        
    def readable(self):
        return True
    
    def readinto(self, b):
        resp = requests.get(self.path, params = {"op": "OPEN", "offset": self.offset, "length": len(b)})
        if self.offset > self.length:
            return 0
        if resp.status_code != 200:
            b[:1] = b"\n"
            self.offset += 1048576 - (self.offset % 1048576)
            return 1
        b[:len(resp.content)]=resp.content
        self.offset += len(resp.content)
        return len(resp.content) # TODO

In [9]:
single=0
multi=0
counter = 0
import time as time
t1=time.time()
for line in io.BufferedReader(hdfsFile("single.csv"),buffer_size=524288):
    line = str(line, "utf-8")
    counter+=1
    if counter>1:
        if 'Single Family' in line:
            single+=1
        elif 'Multifamily' in line:
            multi+=1
t2=time.time()
print('Counts from single.csv with buffer size 512KB')
print('Single Family: ',single)
print('Multi Family: ',multi)
print('Seconds: ',t2-t1)

Counts from single.csv with buffer size 512KB
Single Family:  444874
Multi Family:  2493
Seconds:  21.319695234298706


In [10]:
double=0
multi=0
counter = 0
import time as time
t1=time.time()
for line in io.BufferedReader(hdfsFile("double.csv"),buffer_size=1048576):
    line = str(line, "utf-8")
    counter+=1
    if counter>1:
        if 'Single Family' in line:
            double+=1
        elif 'Multifamily' in line:
            multi+=1
t2=time.time()
print('Counts from double.csv with buffer size 1MB')
print('Single Family: ',double)
print('Multi Family: ',multi)
print('Seconds: ',t2-t1)

Counts from double.csv with buffer size 1MB
Single Family:  444874
Multi Family:  2493
Seconds:  9.27297830581665


# PART 4

In [11]:
!hdfs dfsadmin -fs hdfs://main:9000/ -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 12281319649 (11.44 GB)
DFS Remaining: 12025724928 (11.20 GB)
DFS Used: 255594721 (243.75 MB)
DFS Used%: 2.08%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 92
	Missing blocks (with replication factor 1): 92
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.18.0.2:9866 (project-3-procrastinators-worker-1.cs544net)
Hostname: 87931e77d04b
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 255594721 (243.75 MB)
Non DFS Used: 13522956063 (12.59 GB)
DFS Remaining: 12025724928 (11.20 GB)
DFS Used%: 0.99%
DFS Remain

In [13]:
single=0
multi=0
counter = 0
import time as time
t1=time.time()
for line in io.BufferedReader(hdfsFile("single.csv"),buffer_size=1048576):
    line = str(line, "utf-8")
    counter+=1
    if counter>1:
        if 'Single Family' in line:
            single+=1
        elif 'Multifamily' in line:
            multi+=1
t2=time.time()
print('Counts from single.csv with buffer size 1MB')
print('Single Family: ',single)
print('Multi Family: ',multi)
print('Seconds: ',t2-t1)

Counts from single.csv with buffer size 1MB
Single Family:  199717
Multi Family:  1272
Seconds:  3.2457287311553955


In [14]:
double=0
multi=0
counter = 0
import time as time
t1=time.time()
for line in io.BufferedReader(hdfsFile("double.csv"),buffer_size=1048576):
    line = str(line, "utf-8")
    counter+=1
    if counter>1:
        if 'Single Family' in line:
            double+=1
        elif 'Multifamily' in line:
            multi+=1
t2=time.time()
print('Counts from double.csv with buffer size 1MB')
print('Single Family: ',double)
print('Multi Family: ',multi)
print('Seconds: ',t2-t1)

Counts from double.csv with buffer size 1MB
Single Family:  444874
Multi Family:  2493
Seconds:  4.412144899368286
